In [14]:
from fastai.vision.all import *
from fastai.vision.widgets import *

ModuleNotFoundError: ignored

In [ ]:
path = Path()
learn_inf = load_learner(path/'export.pkl')
btn_upload = widgets.FileUpload()
out_pl = widgets.Output()
lbl_pred = widgets.Label()
btn_run = widgets.Button(description='Classify')

In [ ]:
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'


In [ ]:
btn_run.on_click(on_click_classify)

In [ ]:
VBox([widgets.Label('Select shoes!'), btn_upload, btn_run, out_pl, lbl_pred])